**In this notebook, we use keras_ocr to find the bbox coordinates of the required text recognizd from the image files.**

**Process:**

1. Perform ocr in image file
2. Set the required set of keywords in a list
3. Check if the extracted text matched the set keywords(invoice_number, issue_date, total amount, description table) we require in order to find the bbox coordinates of them
4.For each image file, observe  which text is recognized from the keywords set
5. Only select the top left and bottom right coordinates which is indexed as 0 and 3 in the bbox coordinates returned by keras ocr (found in code below)
6. Append to the list of data fields required
7. Perform this process for all 41 files
8. Make a final bounding box datframe of all the image files

**INSTALLING DEPENDENCIES**

In [ ]:
!pip install keras-ocr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import pandas as pd
import numpy as np

**UPLOADING FILES USING GOOGLE COLAB INTERFACE**

There are 41 jpeg, jpg, png files in the given test dataset which are uploaded and saved using the easy google colab interface. Further processing is done after uploading

In [ ]:
# Upload an image file using the Colab interface
uploaded = files.upload()

Saving 0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg to 0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg
Saving 2a8677b9-b29e-4c93-86e5-56d4c38cb7fc.jpg to 2a8677b9-b29e-4c93-86e5-56d4c38cb7fc.jpg
Saving 2ec7883e-dafe-4cc3-9836-7314ace98c14.png to 2ec7883e-dafe-4cc3-9836-7314ace98c14.png
Saving 3e2ef304-3cb7-4452-87b8-102bea1c2908.jpg to 3e2ef304-3cb7-4452-87b8-102bea1c2908.jpg
Saving 4cfc9619-5dd4-4307-90b6-04d20ba6db3b.jpg to 4cfc9619-5dd4-4307-90b6-04d20ba6db3b.jpg
Saving 4d972974-e804-4c67-af7c-071b16d1d35a.png to 4d972974-e804-4c67-af7c-071b16d1d35a.png
Saving 6a620b62-167a-466e-961d-63cac09ba563.jpg to 6a620b62-167a-466e-961d-63cac09ba563.jpg
Saving 6fab0ac5-e856-43af-a2ed-621d1a15818a.png to 6fab0ac5-e856-43af-a2ed-621d1a15818a.png
Saving 7b450c61-a83d-4843-8404-0828b0d62891.jpg to 7b450c61-a83d-4843-8404-0828b0d62891.jpg
Saving 8e00367b-25b3-401b-bf8e-e1cfc801e4ae.png to 8e00367b-25b3-401b-bf8e-e1cfc801e4ae.png
Saving 9b650c90-ad5c-4f07-b04a-b58e33318398.jpg to 9b650c90-ad5c-4f07-b04a-b58e3

Filenames are extracted for observation, and saved if further processing requires its use

In [ ]:
file_names = pd.DataFrame(uploaded.keys())


In [ ]:
file_names.head()

,0
0,0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg
1,2a8677b9-b29e-4c93-86e5-56d4c38cb7fc.jpg
2,2ec7883e-dafe-4cc3-9836-7314ace98c14.png
3,3e2ef304-3cb7-4452-87b8-102bea1c2908.jpg
4,4cfc9619-5dd4-4307-90b6-04d20ba6db3b.jpg


In [ ]:
csv_file_path = 'filenames.csv'
file_names.to_csv(csv_file_path, index=False)

**Using keras_ocr**

**Main code**

In [ ]:
#firstly, 5 lists of required data fields are created to extract data information from the dataframe
invoice_number = []
invoice_date = []
total = []
description = []
file = []

#iterating through each of the uploaded image file
for filename in uploaded.keys():
    dfs = []
    img = cv2.imread(filename)                                        #reading image
    results = pipeline.recognize([img])                               # using keras pipeline to recognize images which returns bbox and text value recognized by the model
    result = results[0]                                               # selecting data that contains bbox coordinates and text value in form of tuple
    file.append(filename)                                             # appending the name to file[] list which is later used to create a dataframe


    # Check if the result contains text corresponding to 'invoice', 'date', 'total', and 'description'
    found_keywords = []

    #iterating through the tuple obtained through keras_ocr
    for text, bbox_array in result:

        # Check if the ocred text is one of the specified keywords to extract bbox coordinates
        if text.lower() in ['invoice', 'date', 'total', 'description','goods','amount','number','quantity','driver','delivery','company','power'      # here, we check the image files in test set provided and find words that occur
                            'number', 'no.', 'final','printed', 'issued','table','qty', 'amt','delivery','code','pieces','address'                    #  in the invoice which will help in detecting the bbox coordinates. This threw alot of errors
                                                                                                                                                      # because of the diverse structure of the data, so we manually looked at the data and added
                           ,'signature','acceptance', 'No.', 'name'] and text.lower() not in found_keywords and len(found_keywords) != 4:             # keywords present in that file.
                                                                                                                                                      # we have to extract 4 data fields values, hence the condition


            # Extract bounding box coordinates if keras recognized the word and extracted the bbox coordinates and text
            bbox_values = bbox_array  # Convert NumPy array to list

            # Create a DataFrame for the current row of the required data field like invoice number, date, total and table description.
            df = pd.DataFrame({'text': [text], 'bbox': [bbox_values], 'title' : [filename]})
            found_keywords.append(text.lower())                                                                                                        # Mark the keyword as found

            dfs.append(df)                                                                                                                             # append the extracted dataframe to the dfs[] list

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)                                                                                                       # making final dataframe from above
    df_sorted = final_df.sort_values(by = 'text')                                                                                                      # sorting the text values to maintain uniformity

    print(df_sorted)                                                                                                                                   #observation

    # Ectracting bbox coordinates of the required data fields
    coordinates = final_df.iloc[0]['bbox']                                                                                                             #from observation, we saw that index 0 row of the df contains issue date
    #print(coordinates)
    issue_date_bbox = np.concatenate((coordinates[0], coordinates[3]))
    invoice_date.append(issue_date_bbox)
    #print(issue_date_bbox)

    coordinates = final_df.iloc[1]['bbox']                                                                                                             #from observation, we saw that index 1 row of the df generally contains table keyword
    table_bbox = np.concatenate((coordinates[0], coordinates[3]))
    description.append(table_bbox)
    #print(table_bbox)

    coordinates = final_df.iloc[2]['bbox']                                                                                                            #from observation, we saw that index 2 row of the df contains total keyword
    total_bbox = np.concatenate((coordinates[0], coordinates[3]))
    total.append(total_bbox)
    #print(total_bbox)

    coordinates = final_df.iloc[0]['bbox']                                                                                                            #from data observation, we saw that genrally invoice number is genrally present near issue date
    invoice_number_bbox = np.concatenate((coordinates[0], coordinates[2]))                                                                            # this is done because it threw alot of errors since same values for keys aren't present in all invoices
    invoice_number.append(invoice_number_bbox)
    #print(invoice_number_bbox)





10/10 [==============================] - 1s 61ms/step
       text                                               bbox  \
2      code  [[256.28906, 215.15625], [308.4961, 215.15625]...   
1      date  [[653.3789, 151.875], [689.7656, 151.875], [68...   
0   invoice  [[874.8633, 107.578125], [1010.91797, 107.5781...   
3  quantity  [[816.4582, 787.5049], [883.3392, 789.98193], ...   

                                      title  
2  0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg  
1  0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg  
0  0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg  
3  0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg  
4/4 [==============================] - 0s 56ms/step
          text                                               bbox  \
1         date  [[326.0, 86.0], [347.0, 86.0], [347.0, 96.0], ...   
0     delivery  [[203.0, 19.0], [265.0, 19.0], [265.0, 32.0], ...   
2  description  [[122.0, 273.0], [169.0, 273.0], [169.0, 283.0...   
3        goods  [[180.0, 273.0], [209.0, 273.0], [209.0, 

This is how the output looks like when we process all 41 images. We can see that sometimes when required fields aren't found, we made it to pick up  coordinates of keywords found in the images instead of exact data fields that we require.

Checking if the bbox coordinates are extracted for all the required data fields. It should have 41 elements in the list which is the same as the number of examples we processed

In [ ]:
len(invoice_number)

41

In [ ]:
len(invoice_date)

41

In [ ]:
len(total)

41

In [ ]:
len(description)

41

**Everything looks okay, so, we make a bbox dataframe from the exracted fields with their image names**

In [ ]:
bbox = pd.DataFrame(
    { "title" : uploaded.keys(),
     "invoice_number_bbox" : invoice_number,
      "issue_date_bbox" : invoice_date,
      "total_bbox" : total,
      "table_bbox" : description

    }
)

In [ ]:
bbox

,title,invoice_number_bbox,issue_date_bbox,total_bbox,table_bbox
0,0b6fcb50-b157-4457-b3a3-06779f91b8b8.jpg,"[874.8633, 107.578125, 1010.91797, 137.63672]","[874.8633, 107.578125, 874.8633, 137.63672]","[256.28906, 215.15625, 256.28906, 230.97656]","[653.3789, 151.875, 653.3789, 166.11328]"
1,2a8677b9-b29e-4c93-86e5-56d4c38cb7fc.jpg,"[203.0, 19.0, 265.0, 32.0]","[203.0, 19.0, 203.0, 32.0]","[122.0, 273.0, 122.0, 283.0]","[326.0, 86.0, 326.0, 96.0]"
2,2ec7883e-dafe-4cc3-9836-7314ace98c14.png,"[20.0, 126.0, 105.0, 144.0]","[20.0, 126.0, 20.0, 144.0]","[419.0, 226.0, 419.0, 236.0]","[425.0, 169.0, 425.0, 180.0]"
3,3e2ef304-3cb7-4452-87b8-102bea1c2908.jpg,"[1471.6188, 28.412874, 1661.9728, 77.461525]","[1471.6188, 28.412874, 1470.8884, 74.42844]","[1493.0273, 800.4785, 1493.0273, 825.6621]","[163.69336, 724.92773, 163.69336, 750.1113]"
4,4cfc9619-5dd4-4307-90b6-04d20ba6db3b.jpg,"[521.03284, 128.78465, 581.897, 142.80803]","[521.03284, 128.78465, 520.6985, 141.154]","[108.00001, 138.99998, 108.00001, 167.99998]","[18.0, 139.0, 18.0, 168.0]"
5,4d972974-e804-4c67-af7c-071b16d1d35a.png,"[28.0, 105.0, 59.0, 115.0]","[28.0, 105.0, 28.0, 115.0]","[406.0, 167.0, 406.0, 178.0]","[72.03902, 166.75122, 71.67999, 178.23999]"
6,6a620b62-167a-466e-961d-63cac09ba563.jpg,"[207.57031, 430.42188, 301.8047, 449.52344]","[207.57031, 430.42188, 207.57031, 449.52344]","[831.5547, 560.3125, 831.5547, 571.77344]","[206.29688, 560.3125, 206.29688, 571.77344]"
7,6fab0ac5-e856-43af-a2ed-621d1a15818a.png,"[785.15625, 22.265625, 960.9375, 56.25]","[785.15625, 22.265625, 785.15625, 56.25]","[16.40625, 425.39062, 16.40625, 440.625]","[685.5469, 151.17188, 685.5469, 168.75]"
8,7b450c61-a83d-4843-8404-0828b0d62891.jpg,"[409.0, 112.0, 483.0, 126.0]","[409.0, 112.0, 409.0, 126.0]","[209.0, 408.0, 209.0, 418.0]","[454.0, 161.0, 454.0, 170.0]"
9,8e00367b-25b3-401b-bf8e-e1cfc801e4ae.png,"[625.6787, 51.708984, 704.27637, 69.29004]","[625.6787, 51.708984, 625.6787, 69.29004]","[562.59375, 182.01562, 562.59375, 193.3916]","[539.8418, 89.97363, 539.8418, 100.31543]"


**This is the final bbox dataframe obtained from the keras ocr.**

**Saving the file as csv ...**

In [ ]:
csv_file_path = 'boundingbox.csv'
bbox.to_csv(csv_file_path, index=False)